## Hypothesis Testing

[Hypothesis testing](#Hypothesis)<br/>
[Power Calculation](#Power)<br/>
[Experiment Design](#Experiment)<br/>
[Z-test](#Z)<br/>
[T-test](#T)<br/>
[A/B testing](#AB)<br/>
[Bias Variance](#BV)<br/>

In [1]:
from __future__ import division
import numpy as np

<a id='Hypothesis'></a>
### Hypothesis Testing

- State null hypothesis $H_0$ (typically status quo, no effect)
- Choose a significance level $\alpha$
- Choose and compute appropriate test statistics
- Compute p-value and 'reject' or 'fail to reject' $H_0$

**Type I error** - $\alpha$, reject $H_0$ when it is true (convict innocent)<br/>
**Type II error** - $\beta$, fail to reject $H_0$ when it is false (release guilty)

<img src="img/hypothesis_testing.png" width="300">

|              |             | Actual                    |                          |
|--------------|-------------|---------------------------|--------------------------|
|              |             | $H_0$ true                | $H_0$ false              |
|**Predicted** | $H_0$ true  | Correct / TN / 1-$\alpha$ | Type II / FN / $\beta$   | *fail to reject $H_0$*
|              | $H_0$ false | Type I / FP / $\alpha$    | Correct / TP / 1-$\beta$ | *reject $H_0$*

**One-sample vs Two-sample Tests**

Two-sample:
- Two sets of observations to compare - usually from A/B testing
- The question is whether given statistic is better/different for one set of observations; is click through rate on landing page A better than on landing page B given these observations?

One-sample:
- Only one set of observations is available - compare it against specific statistic value
- We have 23 observations telling us the click through rate is 0.217; can we say our overall click through rate is at least 0.2?

**One-sided vs Two-sided Tests**

Two-sided:
- Reject $H_0$ if test statistic is in upper or lower tail
- Compute p-value using probability of being in either tail

One-sided:
- Reject $H_0$ if test statistic is in the wrong tail (advertising does not decrease sales)
- Compute p-value using probability of being in only one tail

<a id='Power'></a>
### Power Calculation

**power** (sensitivity) - 1-$\beta$, probability of not making type II error<br/>
**significance** level (type I eror) - $\alpha$, probability of rejecting $H_0$ given that it is true

[statsmodels documentation](http://statsmodels.sourceforge.net/devel/stats.html#power-and-sample-size-calculations)

- $N (N_T, N_C)$ - number of observation in treatment and control group
- effect size: $\theta_1 - \theta_0$ (for example 1% for direct mail campaign)
- standard deviation: $\sigma (\sigma_T, \sigma_C)$ (p(1-p) for binomial distribution / click through rate)
- significance: $\alpha$ (good value is 0.05)
- power: $1-\beta$ (good value is 0.8)

Provided four of the five information above, we can calculate the rest.

Ideally we want:
- $N_T = N_C$
- big effect size
- lots of data (bigger spikes in the image above)

In [2]:
import scipy.stats as st
from statsmodels.stats import power

# this function was provided in class
def calc_min_sample_size(std_dev, mean_desired, mean_original, alpha=0.05, power=0.8):
    Z_alpha = st.norm.ppf(alpha)
    Z_power = st.norm.ppf(power)

    return ((Z_power - Z_alpha) * std_dev / (mean_desired - mean_original)) ** 2

print 'min sample size: {}'.format(calc_min_sample_size(std_dev=2.6, mean_desired=7.35, 
                                                        mean_original=7.05, alpha=0.05, power=0.8))

from statsmodels.stats.power import tt_solve_power # one sample t-test

print 'min sample size: {}'.format(tt_solve_power(effect_size=(7.35-7.05)/2.6, nobs=None, alpha=0.05, 
                                                  power=0.8, alternative='larger'))

min sample size: 464.378743205
min sample size: 465.734515503


In [3]:
# power calculation in statsmodels
# whatever is set to None is calculated

from statsmodels.stats.power import zt_ind_solve_power # two sample z-test

print 'min sample in group 1: {}'.format(zt_ind_solve_power(effect_size=(7.35-7.05)/2.6, nobs1=None, 
                                                            alpha=0.05, power=0.8, ratio=1, alternative='two-sided'))

from statsmodels.stats.power import tt_ind_solve_power # two sample t-test

print 'min sample in group 1: {}'.format(tt_ind_solve_power(effect_size=(7.35-7.05)/2.6, nobs1=None,
                                                            alpha=0.05, power=0.8, ratio=1, alternative='two-sided'))

min sample in group 1: 1179.0732676
min sample in group 1: 1180.03441727


### Confidence Interval

*If we compute CI from multiple random samples from population, then 95% will contain the true population value.*

- $CI^{(1-\alpha)}$ .. confidence interval<br/>
- $\alpha$ .. significance level<br/>
- $CI^{95} => \alpha = 0.05$

$Prob[truevalue \notin CI^{(1-\alpha)}] \le \alpha$ 

### P-value

*Probability of observing data which is at least as extreme as what was observed assuming the $H_0$ is true.*

<a id='Experiment'></a>
### Experiment Design

Rubin causal model

Classical Random Experiment Assumptions:

1. Individualistic - assignment of person A to treatment not affected bz assignment of person B
- Probabilistic - everyone has non-zero probability of being assigned to each group
- Unconfounded - assignment is independent on outcomes
- Known assignment to treatment
- SUTVA - same treatment for everyone (stable unit treatment value assumption)

Without point 4 the data is **observational**.

Refresher:
- $\sigma^2$ .. variance
- $\sigma$ .. standard deviation
- $\sigma / \sqrt{N}$ .. standard error
- $\mu$ .. mean / average

<a id='Z'></a>
### Z-test

*Use when variance is known.*

$z = \frac {(\overline{x} - \mu)} {\frac {\sigma} {\sqrt{N}}}$

By definition, we know variance of Binomial distribution if we know p:
- Prob(X) = p
- Var = $\sigma^2$ = ${p(1-p)}$
- In cases like click through rate z-test is appropriate: $z = \frac {\mu_1 - \mu_2} {\sqrt{\frac {p_1(1-p_1)} {N_1} + \frac {p_2(1-p_2)} {N_2}}}$

To compare means (assuming they are independent) with the same standard deviation `statsmodels.ztest`:

In [4]:
from statsmodels.stats.weightstats import ztest

A = np.array([0,1,1,1,0,1])
B = np.array([0,1,1,0,0,1])
pA = A.mean()
pB = B.mean()
effect = A.mean() - B.mean()

error = np.sqrt( ((pA*(1-pA))/len(A)) + ((pB*(1-pB))/len(B)) )
z = effect / error

print 'ddof = 0'
print 'my z-test result: {}'.format(z) # 0.594
print 'statsmodels z-test result: {}'.format(ztest(A, B, alternative='two-sided', ddof=0)[0]) # 0.594

error = np.sqrt( ((pA*(1-pA))/(len(A)-1)) + ((pB*(1-pB))/(len(B)-1)) )
z = effect / error

print '\nddof = 1' # default in statsmodels
print 'my z-test result: {}'.format(z) # 0.542
print 'statsmodels z-test result: {}'.format(ztest(A, B, alternative='two-sided', ddof=1)[0]) # 0.542

ddof = 0
my z-test result: 0.594088525786
statsmodels z-test result: 0.594088525786

ddof = 1
my z-test result: 0.542326144547
statsmodels z-test result: 0.542326144547


Statsmodels z-test has three alternatives:
- two sided: the result is **p**
- smaller: one-sided, the result is **p/2**
- larger: one sided, the result is **1-(p/2)**

In [5]:
from statsmodels.stats.weightstats import ztest

print zip(('z-statistic', 'p-value'), ztest([2,3,4,5], [3,4,5,6], alternative='two-sided'))
print zip(('z-statistic', 'p-value'), ztest([2,3,4,5], [3,4,5,6], alternative='smaller'))
print zip(('z-statistic', 'p-value'), ztest([2,3,4,5], [3,4,5,6], alternative='larger'))

[('z-statistic', -1.0954451150103321), ('p-value', 0.27332167829229814)]
[('z-statistic', -1.0954451150103321), ('p-value', 0.13666083914614907)]
[('z-statistic', -1.0954451150103321), ('p-value', 0.86333916085385098)]


To compare means from distributions with different standard deviation use `CompareMeans.ztest_ind`:

In [6]:
from statsmodels.stats.weightstats import CompareMeans, DescrStatsW

cm = CompareMeans(DescrStatsW([2,3,4,5]), DescrStatsW([3,4,5,6]))
print zip(('z-statistic', 'p-value'), cm.ztest_ind(alternative='two-sided'))

[('z-statistic', -1.0954451150103321), ('p-value', 0.27332167829229814)]


<a id='T'></a>
## T-test

*Use when variance is unknown.*

$t = \frac {(\overline{x} - \mu)} {\frac {S} {\sqrt{N}}}$

Used for anything continuous like amount of money spent.

Calculate the T-test for the means of two independent samples of scores:

In [7]:
from scipy.stats import ttest_ind
print ttest_ind([0,0,1,0,2], [0,1,1,0,2], equal_var=False) # Welch’s t-test (do not assume equal population variance)
print ttest_ind([0,0,1,0,2], [0,1,1,0,2], equal_var=True)  # standard independent 2 sample t-test

Ttest_indResult(statistic=-0.36514837167011088, pvalue=0.72450697149417942)
Ttest_indResult(statistic=-0.36514837167011083, pvalue=0.72446582573474294)


<a id='AB'></a>
### A/B Testing

Can we say if one of the landing pages is better (i.e. gets more registrations) than the other with statistical significance?

|                 | Visitors  | Registrations |
|-----------------|-----------|---------------|
| Landing Page A  | 1,012     | 349           |
| Landing Page B  |   995     | 320           |

- $H_0$: Landing pages A and B are the same.
- $H_1$: One of the pages is better.

Visit > register ratio
- Landing Page A: 349 / 1,012 = 0.3448
- Landing Page B: 320 / 995 = 0.3216

In [8]:
# create data: visit without registering 0, visit and register 1
lpA = np.zeros(1012)
lpA[0:349] = 1
lpB = np.zeros(995)
lpB[0:320] = 1

from statsmodels.stats.weightstats import ztest

print zip(('z-statistic', 'p-value'), ztest(lpA, lpB, alternative='two-sided'))
print zip(('z-statistic', 'p-value'), ztest(lpA, lpB, alternative='larger'))

[('z-statistic', 1.1046903609737513), ('p-value', 0.26929378150307026)]
[('z-statistic', 1.1046903609737513), ('p-value', 0.13464689075153513)]


In [9]:
from scipy.stats import chi2_contingency

chi2, p, dof, expected = chi2_contingency([[349, 1012-349], [320, 995-320]], correction=False)
print p, p/2

# do we need to Yates contiuity correction here?

0.269200758683 0.134600379342


<a id='BV'></a>
### Bias-variance Tradeoff

Bias and Variance cannot actually be calculated!
- Bias = underfitting
- Variance = overfitting

$ Bias = E[\hat{f}(x)] - f(x) $ ... model prediction - truth

$ Variance = E[ (\hat{f}(x) - E[\hat{f}(x)])^{2} ] $

$ MSE = Bias^{2} + ModelVariance + \sigma_\epsilon^{2} $ ... sigma is irreducible error

- Inference is possible with low (almost zero) bias.
- Gauss–Markov theorem: OLS is BLUE = ordinary least square estimator is best linear unbiased estimator